In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import ast
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies = movies.merge(credits,on='title')


movies_feature_set = movies[['id','title','keywords','genres','cast','popularity','vote_average','vote_count']]
movies_feature_set.dropna(inplace=True)
movies_feature_set.head()


/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,id,title,keywords,genres,cast,popularity,vote_average,vote_count
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",150.437577,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",139.082615,6.9,4500
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",107.376788,6.3,4466
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",112.312950,7.6,9106
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...",43.926995,6.1,2124


In [3]:


def convert(text):
    arr = []
    for i in ast.literal_eval(text):
        arr.append(i['name'])
    
    return arr

movies_feature_set['genres'] = movies['genres'].apply(convert)


def convert_keyword_top3(text):
    arr= []
    count = 0
    for i in ast.literal_eval(text):
        if count<3:
            arr.append(i['name'])
            count+=1
    return arr   

movies_feature_set['keywords'] = movies['keywords'].apply(convert_keyword_top3)

def convert_cast_top2(text):
    arr= []
    count = 0
    for i in ast.literal_eval(text):
        if count<2:
            arr.append(i['name'])
            count+=1
    return arr        
movies_feature_set['cast'] = movies['cast'].apply(convert_cast_top2)
movies_feature_set['tags'] = movies_feature_set['keywords'] + movies_feature_set['genres'] + movies_feature_set['cast']
new_df = movies_feature_set[['id','title','tags']]
new_df.head(5)




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,id,title,tags
0,19995,Avatar,"[culture clash, future, space war, Action, Adv..."
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, Adventure, ..."
2,206647,Spectre,"[spy, based on novel, secret agent, Action, Ad..."
3,49026,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, Action, ..."
4,49529,John Carter,"[based on novel, mars, medallion, Action, Adve..."


In [4]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head(3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,title,tags
0,19995,Avatar,culture clash future space war Action Adventur...
1,285,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island Adventure Fanta...
2,206647,Spectre,spy based on novel secret agent Action Adventu...


In [5]:
new_df['tags'][0]

'culture clash future space war Action Adventure Fantasy Science Fiction Sam Worthington Zoe Saldana'

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
new_df['overview'] = new_df['tags'].fillna('')
tfidf_matrix = tfidf.fit_transform(new_df['tags'])

tfidf_matrix.shape


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(4809, 7526)

In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(new_df.index, index=new_df['title']).drop_duplicates()

indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4804
Newlyweds                                   4805
Signed, Sealed, Delivered                   4806
Shanghai Calling                            4807
My Date with Drew                           4808
Length: 4809, dtype: int64

In [8]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie related to its title 
    idx = indices[title]

    # pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sorting of movies based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar movies
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar movies
    return new_df['title'].iloc[movie_indices]




In [9]:
get_recommendations('The Avengers')

26              Captain America: Civil War
7                  Avengers: Age of Ultron
79                              Iron Man 2
85     Captain America: The Winter Soldier
169     Captain America: The First Avenger
Name: title, dtype: object